In [1]:
%autoreload 2

In [2]:
from argparse import Namespace
from collections import defaultdict
import copy
from datetime import datetime
import difflib
import gzip
import itertools
import os
import pickle
import sys
import typing

from IPython.display import display, Markdown, HTML  # type: ignore
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import swifter
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline
import tatsu
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm.notebook as tqdmn


sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src import fitness_energy_utils as utils
from src.ast_mcmc_regrowth import _load_pickle_gzip
from src.fitness_features import *
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.latest_model_paths import LATEST_FITNESS_FEATURIZER_PATH, LATEST_FITNESS_FUNCTION_DATE_ID, LATEST_FITNESS_FEATURES
from src.ast_counter_sampler import *
from src.ast_utils import cached_load_and_parse_games_from_file, load_games_from_file, _extract_game_id
from src import ast_printer
from src.fitness_features_preprocessing import NGRAM_SCORE_PATTERN
from src.evolutionary_sampler import *

2023-05-14 18:46:42 - ast_utils - DEBUG    - Using cache folder: /misc/vlgscratch4/LakeGroup/guy/game_generation_cache
2023-05-14 18:46:43 - src.ast_utils - DEBUG    - Using cache folder: /misc/vlgscratch4/LakeGroup/guy/game_generation_cache
2023-05-14 18:46:43 - git.cmd - DEBUG    - Popen(['git', 'version'], cwd=/home/gd1279/projects/game-generation-modeling/notebooks, universal_newlines=False, shell=None, istream=None)
2023-05-14 18:46:43 - git.cmd - DEBUG    - Popen(['git', 'version'], cwd=/home/gd1279/projects/game-generation-modeling/notebooks, universal_newlines=False, shell=None, istream=None)


In [3]:
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
real_game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='..'))  # type: ignore

FEATURIZER_PATH = LATEST_FITNESS_FEATURIZER_PATH
MODEL_DATE_ID = LATEST_FITNESS_FUNCTION_DATE_ID
MODEL_NAME = DEFAULT_SAVE_MODEL_NAME
MAP_ELITES_PATH_SPECS = [
    ('2023_05_13', 'samples', 'evo_sampler_map_elites_fitness_rank_node_count_objects_setup_final'),
    ('2023_05_13', 'samples', 'evo_sampler_map_elites_fitness_rank_node_count_predicates_setup_final'),
    ('2023_05_13', 'samples', 'evo_sampler_map_elites_ucb_node_count_objects_setup_final'),
    ('2023_05_13', 'samples', 'evo_sampler_map_elites_ucb_node_count_predicates_setup_final'),   
]

fitness_featurizer = _load_pickle_gzip(FEATURIZER_PATH)
fitness_function, feature_names = load_model_and_feature_columns(MODEL_DATE_ID, name=MODEL_NAME, relative_path='..')  # type: ignore
original_fitness_function, _ = load_model_and_feature_columns(MODEL_DATE_ID, name=MODEL_NAME, relative_path='..')  # type: ignore

In [4]:
DEFAULT_MIN_FITNESS = 0.0

def load_map_elites_games(map_elites_path_specs: typing.List[typing.Tuple[str, str, str]], min_fitness: float = DEFAULT_MIN_FITNESS):
    map_elites_games = []
    for date_id, folder, filename in map_elites_path_specs:
        map_elites = typing.cast(MAPElitesSampler, utils.load_data(date_id, folder, filename))
        for key in map_elites.population:
            if map_elites.fitness_values[key] > min_fitness:
                map_elites_games.append((map_elites.population[key], filename))
        
    return map_elites_games


MAP_ELITES_GAMES = load_map_elites_games(MAP_ELITES_PATH_SPECS)

In [5]:
fitness_df = utils.load_fitness_data(LATEST_FITNESS_FEATURES)
print(fitness_df.src_file.unique())
print(fitness_df.shape)

fitness_featurizer.preprocessors[1].forced_output_keys = set([c for c in fitness_df.columns if 'arg_types' in c])

['interactive-beta.pddl' 'ast-real-regrowth-samples-1024.pddl']
(100450, 391)


In [6]:
def create_fitness_df_with_map_elites_games(
    current_fitness_df: pd.DataFrame,
    fitness_featurizer: ASTFitnessFeaturizer,
    real_games: typing.List,
    map_elites_games: typing.List[tuple], 
):

    for game in real_games:
        fitness_featurizer.parse(game, 'interactive-beta.pddl')

    for game, filename in tqdmn.tqdm(map_elites_games):
        fitness_featurizer.parse(game, filename)

    map_elites_fitness_df = fitness_featurizer.to_df(use_prior_values=[True, False])
    return utils.process_fitness_df(pd.concat([map_elites_fitness_df], ignore_index=True))

In [7]:
map_elites_fitness_df = create_fitness_df_with_map_elites_games(fitness_df, fitness_featurizer, real_game_asts, MAP_ELITES_GAMES)
print(map_elites_fitness_df.shape, fitness_df.shape)

unexpected_columns = set(map_elites_fitness_df.columns) - set(fitness_df.columns)
missing_columns = set(fitness_df.columns) - set(map_elites_fitness_df.columns)

if len(unexpected_columns) > 0:
    raise ValueError(f'Unexpected columns: {unexpected_columns}')

if len(missing_columns) > 0:
    print(f'Missing columns: {missing_columns}')
    map_elites_fitness_df = map_elites_fitness_df.reindex(columns=fitness_df.columns, fill_value=0.0)

missing_columns = set(fitness_df.columns) - set(map_elites_fitness_df.columns)
if len(missing_columns) > 0:
    raise ValueError(f'Failed to fill missing columns: {missing_columns}')

print(map_elites_fitness_df.real.value_counts())
print(map_elites_fitness_df.isna().sum().sum())

  0%|          | 0/743 [00:00<?, ?it/s]

2023-05-14 18:47:07 - root - INFO     - No features found for prefix agent_holds_arg_types and suffix setup
2023-05-14 18:47:07 - root - INFO     - No features found for prefix in_motion_arg_types and suffix setup
2023-05-14 18:47:07 - root - INFO     - No features to merge for prefix object_orientation_arg_types and suffix setup
2023-05-14 18:47:07 - root - INFO     - No features found for prefix touch_arg_types and suffix setup
2023-05-14 18:47:07 - root - INFO     - No features found for prefix adjacent_side_3_arg_types and suffix constraints
2023-05-14 18:47:07 - root - INFO     - No features to merge for prefix agent_holds_arg_types and suffix constraints
2023-05-14 18:47:07 - root - INFO     - No features found for prefix between_arg_types and suffix constraints
2023-05-14 18:47:07 - root - INFO     - No features to merge for prefix distance_arg_types and suffix constraints
2023-05-14 18:47:07 - root - INFO     - No features to merge for prefix in_arg_types and suffix constraints

(841, 374) (100450, 391)
Missing columns: {'in_arg_types_other_constraints', 'in_arg_types_balls_balls_constraints', 'agent_holds_arg_types_furniture_constraints', 'object_orientation_arg_types_other_constraints', 'agent_holds_arg_types_agent_constraints', 'Index', 'in_arg_types_receptacles_receptacles_constraints', 'object_orientation_arg_types_other_setup', 'distance_arg_types_other_constraints', 'on_arg_types_balls_balls_constraints', 'on_arg_types_receptacles_receptacles_constraints', 'agent_holds_arg_types_other_constraints', 'in_arg_types_blocks_blocks_constraints', 'agent_holds_arg_types_room_features_constraints', 'in_motion_arg_types_agent_constraints', 'in_motion_arg_types_other_constraints', 'in_motion_arg_types_room_features_constraints'}
0    743
1     98
Name: real, dtype: int64
0


## TODO:
* Split this dataset into train and test
* Fine-tune the existing energy function on this (probably adjusting the value of `k`)
* Evaluate the fine-tuned model
* Print the changes in feature weights?

In [8]:
RANDOM_SEED = DEFAULT_RANDOM_SEED 


data_tensor, _ = utils._input_data_to_train_test_tensors(input_data=map_elites_fitness_df, feature_columns=feature_names,
        split_test_set=False, random_seed=RANDOM_SEED, train_prop=utils.DEFAULT_TRAINING_PROP, ignore_original_game=True)

train_tensor, test_tensor = utils.train_test_split(data_tensor, random_state=RANDOM_SEED,
                train_size=utils.DEFAULT_TRAINING_PROP)


if test_tensor is not None:
    print(f'Train tensor shape: {train_tensor.shape} | Test tensor shape: {test_tensor.shape}')  # type: ignore
else:
    print(f'Train tensor shape: {train_tensor.shape}')

k = train_tensor.shape[1] - 1
print(f'Using k={k} for fitness function')
fitness_function.named_steps['fitness'].train_kwargs['k'] = k  # type: ignore
fitness_function.named_steps['fitness'].train_kwargs['device'] = 'cpu'  # type: ignore

fine_tuned_fitness_function = fitness_function.fit(train_tensor)

Train tensor shape: torch.Size([78, 8, 366]) | Test tensor shape: torch.Size([20, 8, 366])
Using k=7 for fitness function


In [9]:
print('Before tuning, training set:')
print(utils.default_multiple_scoring(original_fitness_function, train_tensor))

print('After tuning, training set:')
print(utils.default_multiple_scoring(fitness_function, train_tensor))

if test_tensor is not None:
    print('Before tuning, test set:')
    print(utils.default_multiple_scoring(original_fitness_function, test_tensor))

    print('After tuning, test set:')
    print(utils.default_multiple_scoring(fitness_function, test_tensor))


Before tuning, training set:
{'loss': -1.578116536140442, 'overall_ecdf': -0.19241042077580536, 'single_game_rank': 0.8443223834037781, 'single_game_min_rank': 0.4285714328289032, 'energy_of_negative@1%': 79.01666259765625, 'energy_of_negative@5%': 76.26824951171875}
After tuning, training set:
{'loss': -0.015177130699157715, 'overall_ecdf': -0.06332182774490468, 'single_game_rank': 1.0, 'single_game_min_rank': 1.0, 'energy_of_negative@1%': -0.9931230545043945, 'energy_of_negative@5%': -2.4131953716278076}
Before tuning, test set:
{'loss': -1.6508722305297852, 'overall_ecdf': -0.19437500000000002, 'single_game_rank': 0.8642856478691101, 'single_game_min_rank': 0.5714285969734192, 'energy_of_negative@1%': 79.46068572998047, 'energy_of_negative@5%': 77.41535186767578}
After tuning, test set:
{'loss': -0.007171182427555323, 'overall_ecdf': -0.065625, 'single_game_rank': 1.0, 'single_game_min_rank': 1.0, 'energy_of_negative@1%': -0.7277530431747437, 'energy_of_negative@5%': -2.241564273834

In [11]:
original_data_tensor, _ = utils._input_data_to_train_test_tensors(input_data=fitness_df, feature_columns=feature_names,
        split_test_set=False, random_seed=RANDOM_SEED, train_prop=utils.DEFAULT_TRAINING_PROP, ignore_original_game=True)

print(original_data_tensor.shape)

print('Before tuning, original full dataset:')
print(utils.default_multiple_scoring(original_fitness_function, original_data_tensor))

print('After tuning, original full  set:')
print(utils.default_multiple_scoring(fitness_function, original_data_tensor))

torch.Size([98, 1025, 366])
Before tuning, original full dataset:
{'loss': -3.5341789722442627, 'overall_ecdf': -0.009690982415863308, 'single_game_rank': 0.9801897406578064, 'single_game_min_rank': 0.9296875, 'energy_of_negative@1%': 74.36898803710938, 'energy_of_negative@5%': 72.7696762084961}
After tuning, original full  set:
{'loss': -6.980717658996582, 'overall_ecdf': -0.44103676313731066, 'single_game_rank': 0.6973652839660645, 'single_game_min_rank': 0.3642578125, 'energy_of_negative@1%': 7.561910152435303, 'energy_of_negative@5%': 6.620471954345703}
